In [1]:
import pandas as pd
import numpy as np
import time

#!pip install xgboost
#!pip install lightgbm

In [2]:

# GitHub raw URL (important: replace %20 with spaces or use raw link)
url = "https://raw.githubusercontent.com/artem-ai-blip/Data-Analysis-3/main/Assignment%202/Crete,%202024,%20Q1.csv"

# Load the dataset
crete_df = pd.read_csv(url)

# Display basic info
crete_df.info()
crete_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25772 entries, 0 to 25771
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              25772 non-null  object 
 1   name                            21603 non-null  object 
 2   host_id                         21602 non-null  float64
 3   host_name                       21602 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   21602 non-null  object 
 6   latitude                        21602 non-null  float64
 7   longitude                       21602 non-null  float64
 8   room_type                       21602 non-null  object 
 9   price                           21077 non-null  float64
 10  minimum_nights                  21602 non-null  float64
 11  number_of_reviews               21602 non-null  float64
 12  last_review                     

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license;
0,27966,Heraklion-Pinelopi Apartment,120502.0,Emmanouil,NaN,Μαλεβιζίου,35.33012,25.08012,Entire home/apt,45.0,2.0,128.0,2023-09-20,0.82,2.0,334.0,5.0,00000247117;
1,"29130,""Villa Kallergi - Athena, 12 guests"",125...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"29849,""Villa Kallergi - Nefeli, 6 guests"",1252...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29856,Matala Dimitris Villa - Four Bed Room,128653.0,Dimitris,NaN,Φαιστού,34.99533,24.75654,Private room,148.0,1.0,44.0,2023-09-20,0.26,4.0,33.0,1.0,00001193488;
4,31023,Chryssoula Guesthouse balcony (200mbps),133208.0,Chryssoula,NaN,Χανίων,35.51439,24.01793,Entire home/apt,61.0,2.0,313.0,2024-06-23,2.15,4.0,204.0,12.0,1246760;


neighbourhood_group has only nulls — we’ll likely drop this.

price has ~2,695 missing values.

last_review and reviews_per_month are missing for about 1/4 of the data — we can impute or engineer features from that.

license; has a semicolon in its name — we should fix that column name.

In [3]:
# Clean column names
crete_df.columns = crete_df.columns.str.strip().str.replace(';', '', regex=False)

# Drop columns with all missing values
crete_df = crete_df.drop(columns=['neighbourhood_group'])

# Check missing data
missing = crete_df.isnull().sum()
missing = missing[missing > 0]
print("Missing values:\n", missing)

# Basic imputation strategy
crete_df['price'] = crete_df['price'].fillna(crete_df['price'].median())
crete_df['reviews_per_month'] = crete_df['reviews_per_month'].fillna(0)
crete_df['last_review'] = pd.to_datetime(crete_df['last_review'], errors='coerce')
crete_df['last_review'] = crete_df['last_review'].fillna(pd.Timestamp('2023-01-01'))


Missing values:
 name                              4169
host_id                           4170
host_name                         4170
neighbourhood                     4170
latitude                          4170
longitude                         4170
room_type                         4170
price                             4695
minimum_nights                    4170
number_of_reviews                 4170
last_review                       9871
reviews_per_month                 9871
calculated_host_listings_count    4170
availability_365                  4170
number_of_reviews_ltm             4170
license                           4170
dtype: int64


In [4]:
# Drop unnecessary columns
crete_df = crete_df.drop(columns=['name', 'host_id', 'host_name', 'license'])

# Drop rows with missing values in key columns
crete_df = crete_df.dropna(subset=[
    'neighbourhood', 'latitude', 'longitude', 'room_type',
    'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
    'availability_365', 'number_of_reviews_ltm'
])

# Confirm cleanup
crete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21602 entries, 0 to 25771
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              21602 non-null  object        
 1   neighbourhood                   21602 non-null  object        
 2   latitude                        21602 non-null  float64       
 3   longitude                       21602 non-null  float64       
 4   room_type                       21602 non-null  object        
 5   price                           21602 non-null  float64       
 6   minimum_nights                  21602 non-null  float64       
 7   number_of_reviews               21602 non-null  float64       
 8   last_review                     21602 non-null  datetime64[ns]
 9   reviews_per_month               21602 non-null  float64       
 10  calculated_host_listings_count  21602 non-null  float64       
 11  av

Amendments to the data 

1. Date-Based Features
Extract year/month/day from last_review

Create a "days since last review" feature

2. Log-Transform Skewed Columns
price, minimum_nights, number_of_reviews, reviews_per_month, etc.

3. Categorical Encoding
room_type → One-hot or ordinal encoding

neighbourhood → One-hot (or frequency encoding if too many categories)

4. Interaction Features (Optional)
reviews_per_month * availability_365

number_of_reviews_ltm / number_of_reviews

In [5]:
# 1. Date-based feature
crete_df['days_since_last_review'] = (pd.Timestamp('2024-01-01') - crete_df['last_review']).dt.days

# 2. Log-transform skewed variables
for col in ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month']:
    crete_df[f'log_{col}'] = np.log1p(crete_df[col])

# 3. One-hot encode categorical variables
crete_df = pd.get_dummies(crete_df, columns=['room_type', 'neighbourhood'], drop_first=True)

# Final shape check
print(f"Final dataset shape: {crete_df.shape}")
crete_df.head()


Final dataset shape: (21602, 42)


,id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,...,neighbourhood_Μινώα Πεδιάδας,neighbourhood_Μυλοποτάμου,neighbourhood_Οροπεδίου Λασιθίου,neighbourhood_Πλατανιά,neighbourhood_Ρεθύμνης,neighbourhood_Σητείας,neighbourhood_Σφακίων,neighbourhood_Φαιστού,neighbourhood_Χανίων,neighbourhood_Χερσονήσου
0,27966,35.330120,25.080120,45.0,2.0,128.0,2023-09-20,0.82,2.0,334.0,...,0,0,0,0,0,0,0,0,0,0
3,29856,34.995330,24.756540,148.0,1.0,44.0,2023-09-20,0.26,4.0,33.0,...,0,0,0,0,0,0,0,1,0,0
4,31023,35.514390,24.017930,61.0,2.0,313.0,2024-06-23,2.15,4.0,204.0,...,0,0,0,0,0,0,0,0,1,0
5,31789,35.496480,23.696480,200.0,3.0,2.0,2012-09-23,0.01,1.0,365.0,...,0,0,0,0,0,0,0,0,0,0
6,34280,35.396159,25.025414,85.0,2.0,131.0,2023-09-27,0.97,11.0,31.0,...,0,0,0,0,0,0,0,0,0,0


To build a pricing model for Airbnb listings in Crete, our goal is to accurately predict nightly prices based on listing features. This model supports dynamic pricing strategies and helps optimize revenue. Our first step was variable selection, where we retained features that signal value or influence pricing behavior. Geographic coordinates (latitude, longitude) were included due to the importance of location in determining price, especially proximity to tourist hotspots. Room type and neighbourhood were kept as categorical indicators of accommodation style and socio-geographic pricing patterns. We included minimum_nights as it hints at the stay duration and value proposition. Popularity-related metrics like number_of_reviews, reviews_per_month, and number_of_reviews_ltm act as social proof, while last_review was transformed into a more interpretable metric — days_since_last_review. calculated_host_listings_count helps differentiate between professional and casual hosts, and availability_365 may indicate listing strategy. The target variable is, of course, price. Irrelevant or non-predictive columns like id, name, host_id, host_name, and license were dropped due to their administrative nature.

 To address skewed distributions, we applied log transformations to variables such as price, minimum_nights, and number_of_reviews, enabling better learning for models like OLS or LASSO. Categorical variables like room_type and neighbourhood were one-hot encoded, allowing machine learning algorithms to interpret them correctly. We were mindful of overfitting risks, opting for drop_first=True in one-hot encoding and excluding non-informative identifiers.

In summary, our design choices focused on features related to location, quality, popularity, and availability. Through thoughtful transformation and selection, we ensured interpretability, minimized skew, and improved model robustness — laying a solid foundation for dynamic, data-driven Airbnb pricing.

## Modelling

Before training a machine learning model, it's important to inspect the shape of the input data using .shape. For instance, X_train.shape might return (17,484, 64), indicating 17,484 samples and 64 features in the training set. Similarly, X_test.shape returning (4,372, 64) confirms that the test set includes the same number of features. While not strictly required for the model to function, checking shapes serves as a valuable sanity check — it ensures the train-test split was performed correctly and that the datasets are not empty or imbalanced. 

In [6]:
from sklearn.model_selection import train_test_split

# Drop columns we don’t want in features
drop_cols = ['id', 'last_review', 'price', 'log_price']
X = crete_df.drop(columns=drop_cols)
y = crete_df['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (17281, 38), Test shape: (4321, 38)


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Initialize and fit OLS model
ols_model = LinearRegression()
ols_model.fit(X_train, y_train)

# Predict
y_pred_ols = ols_model.predict(X_test)

# Evaluate
r2_ols = r2_score(y_test, y_pred_ols)
rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))

print(f"OLS R²: {r2_ols:.4f}")
print(f"OLS RMSE: {rmse_ols:.2f}")


OLS R²: 0.0788
OLS RMSE: 477.53


Training on the entire dataset would result in a perfect-fit model with no opportunity to assess its ability to generalize to new or unseen data. The train-test split is essential because it mirrors real-world deployment: the model is trained on known data, and predictions are made on new data (the test set) to evaluate performance. This approach helps prevent overfitting, where a model learns the training data too well and fails to perform effectively on unfamiliar inputs. Modeling flow involves training OLS on X_train and y_train, generating predictions on X_test, and then evaluating the results using metrics such as R² and RMSE. 



In [8]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# We'll use GridSearchCV to find the best alpha (regularization strength)
alphas = np.logspace(-3, 1, 50)
lasso = Lasso(max_iter=10000)

param_grid = {'alpha': alphas}
grid_lasso = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_lasso.fit(X_train, y_train)

# Best model
best_lasso = grid_lasso.best_estimator_
y_pred_lasso = best_lasso.predict(X_test)

# Evaluate
r2_lasso = r2_score(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print(f"LASSO R²: {r2_lasso:.4f}")
print(f"LASSO RMSE: {rmse_lasso:.2f}")
print(f"Best alpha: {grid_lasso.best_params_['alpha']}")


LASSO R²: 0.0785
LASSO RMSE: 477.59
Best alpha: 0.029470517025518096


In [9]:
lasso_coef = pd.Series(best_lasso.coef_, index=X.columns)
print(f"Non-zero coefficients: {(lasso_coef != 0).sum()} / {len(lasso_coef)}")


Non-zero coefficients: 36 / 38


LASSO 

### Random Forest 

In [10]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and fit model
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print(f"Random Forest R²: {r2_rf:.4f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")


Random Forest R²: 0.6074
Random Forest RMSE: 311.73


In [11]:
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)
print(importances.head(10))


longitude                         0.208912
latitude                          0.162713
availability_365                  0.143565
calculated_host_listings_count    0.143203
neighbourhood_Αποκορώνου          0.087578
log_minimum_nights                0.035856
minimum_nights                    0.032907
days_since_last_review            0.026853
neighbourhood_Αγίου Νικολάου      0.022565
log_reviews_per_month             0.018182
dtype: float64


### Boosting 

In [12]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and fit model
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
r2_xgb = r2_score(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

print(f"XGBoost R²: {r2_xgb:.4f}")
print(f"XGBoost RMSE: {rmse_xgb:.2f}")


XGBoost R²: 0.5930
XGBoost RMSE: 317.41


In [13]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the model
lgbm_model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Fit the model
lgbm_model.fit(X_train, y_train)

# Predict
y_pred_lgbm = lgbm_model.predict(X_test)

# Evaluate
r2_lgbm = r2_score(y_test, y_pred_lgbm)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

print(f"LightGBM R²: {r2_lgbm:.4f}")
print(f"LightGBM RMSE: {rmse_lgbm:.2f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2234
[LightGBM] [Info] Number of data points in the train set: 17281, number of used features: 36
[LightGBM] [Info] Start training from score 204.000174
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

## Model Comparison 

In [15]:
# Define models and labels
models = {
    "OLS": LinearRegression(),
    "LASSO": Lasso(alpha=0.03, max_iter=10000),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
}

results = []

# Evaluate each model
for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()
    
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    runtime = end - start

    results.append({
        "Model": name,
        "R²": round(r2, 4),
        "RMSE (€)": round(rmse, 2),
        "Train Time (s)": round(runtime, 4)
    })

# Create table
results_df = pd.DataFrame(results)
results_df.sort_values(by="R²", ascending=False, inplace=True)
print(results_df)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2234
[LightGBM] [Info] Number of data points in the train set: 17281, number of used features: 36
[LightGBM] [Info] Start training from score 204.000174
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [20]:
import plotly.graph_objects as go

# Create table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Model", "R²", "RMSE (€)", "Train Time (s)"],
        fill_color='#2D3748',
        font=dict(color='white', size=14),
        align='center',
        height=30
    ),
    cells=dict(
        values=[
            results_df["Model"],
            results_df["R²"],
            results_df["RMSE (€)"],
            results_df["Train Time (s)"]
        ],
        fill_color='#F7FAFC',
        align='center',
        font=dict(color='black', size=13),
        height=28
    ))
])

fig.update_layout(
    width=700,
    height=350,
    title="Model Horserace: Fit vs Time",
    title_font=dict(size=20)
)

fig.show()


## RF vs XGBoost

In [23]:
# Get feature importances from RF
rf_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
rf_top10 = rf_importances.sort_values(ascending=False).head(10)

print("Random Forest — Top 10 Features")
display(rf_top10)


Random Forest — Top 10 Features


longitude                         0.208912
latitude                          0.162713
availability_365                  0.143565
calculated_host_listings_count    0.143203
neighbourhood_Αποκορώνου          0.087578
log_minimum_nights                0.035856
minimum_nights                    0.032907
days_since_last_review            0.026853
neighbourhood_Αγίου Νικολάου      0.022565
log_reviews_per_month             0.018182
dtype: float64

In [24]:
# Get feature importances from XGBoost
xgb_importances = pd.Series(xgb_model.feature_importances_, index=X.columns)
xgb_top10 = xgb_importances.sort_values(ascending=False).head(10)

print("XGBoost — Top 10 Features")
display(xgb_top10)


XGBoost — Top 10 Features


neighbourhood_Αποκορώνου          0.266950
room_type_Hotel room              0.099420
neighbourhood_Αγίου Νικολάου      0.055139
longitude                         0.053088
calculated_host_listings_count    0.051646
log_minimum_nights                0.048542
availability_365                  0.041290
latitude                          0.038244
log_reviews_per_month             0.035343
room_type_Private room            0.030272
dtype: float32

## Part I, Step 4: Model Interpretation & Feature Comparison

### Top 10 Feature Comparison — Random Forest vs XGBoost

| Rank | Random Forest                    | Importance | XGBoost                          | Importance |
|------|----------------------------------|------------|----------------------------------|------------|
| 1    | **longitude**                    | 0.209      | **neighbourhood_Αποκορώνου**     | 0.267      |
| 2    | **latitude**                     | 0.163      | **room_type_Hotel room**         | 0.099      |
| 3    | availability_365                 | 0.144      | neighbourhood_Αγίου Νικολάου     | 0.055      |
| 4    | calculated_host_listings_count  | 0.143      | longitude                         | 0.053      |
| 5    | neighbourhood_Αποκορώνου        | 0.088      | calculated_host_listings_count   | 0.052      |
| 6    | log_minimum_nights              | 0.036      | log_minimum_nights               | 0.049      |
| 7    | minimum_nights                  | 0.033      | availability_365                 | 0.041      |
| 8    | days_since_last_review          | 0.027      | latitude                         | 0.038      |
| 9    | neighbourhood_Αγίου Νικολάου    | 0.023      | log_reviews_per_month            | 0.035      |
| 10   | log_reviews_per_month           | 0.018      | room_type_Private room           | 0.030      |

---

### Key Insights 

- **Location dominates** both models — longitude and latitude are top predictors, confirming spatial pricing patterns in Crete.
- **Neighbourhoods** such as Αποκορώνου and Αγίου Νικολάου are influential in both models, capturing local demand.
- **Room types** (especially Hotel and Private rooms) are important in XGBoost but not in Random Forest — likely due to better handling of sparse dummies.
- **Availability** and **host listing count** reflect operational/business strategy, influencing pricing.
- **Minimum nights & recent activity** (log_reviews_per_month, days_since_last_review) also affect price, hinting at host flexibility and guest engagement levels.

---
 
### Summary

In comparing feature importance between Random Forest and XGBoost, both models highlight location as a dominant pricing factor, with longitude and latitude consistently ranked at the top. Neighbourhoods such as Αποκορώνου and Αγίου Νικολάου also emerge as key predictors, capturing local demand dynamics. While Random Forest emphasizes spatial and availability-related features, XGBoost assigns more weight to categorical and engineered variables, including room types and log-transformed review metrics—likely due to its superior handling of sparse data. Both models reflect the influence of host strategy (e.g., availability, number of listings) and guest interaction indicators (e.g., days since last review), suggesting that Airbnb pricing in Crete is shaped by a combination of location, listing flexibility, and user engagement.

- **Random Forest** favors spatial and availability-related features.
- **XGBoost** offers a more balanced view, incorporating engineered and categorical variables.
- Both models effectively capture non-linear and complex interactions in Airbnb pricing.


# Part II. Validity